<h1 style="text-align:center;">Project: Simulation of an ideal gas</h1>
<p style="text-align:center;">by Simon Legtenborg, 3773994</p>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import ipywidgets as widgets
from ipycanvas import Canvas, hold_canvas
from time import sleep
from threading import Thread
from scipy.optimize import curve_fit

<h2> Introduction </h2>

<p>Einer der faszinierendsten Themen der Thermodynamik ist das Wechselspiel aus Marko- und Mikrophysik. Deswegen habe ich mir das Ziel gesetzt, eine Simulation zu entwickeln, die ein Gas auf mikroskopischer Ebene simuliert und makroskopische Zustände ausliest. Damit sollen die aus der Vorlesung bereits hergeleiteten Zustandsgleichungen (2. Gay-Lussac'sches Gesetz, Boyle-Mariotte'sches Gesetz, ideale Gasgleichung) überprüft und bestätigt werden. 
Die Arbeit bietet außerdem eine interaktive Simulation. </p>

<h2>Fundamentals</h2>

Bevor wir zur Simulation kommen, müssen die Gesetzmäßigkeiten erklärt, die wir im Anschluss überprüfen wollen.

<h3>Zustandsgleichungen</h3>

<p>In der Simulation kann Volumen, Temperatur und Teilchenzahl variiert bzw. konstant gehalten werden. Es bietet sich deshalb an, die Gesetze von Boyle-Marionette und Gay-Lussac (2. Gesetz) zu überprüfen:</p>

<p><b>Boyle Marionette: </b>Wird eine feste Menge eines idealen Gases auf einer konstanten Temperaur gehalten, während Druck und Volumen veränderbar sind, so ist das Volumen indirekt proportional
zum Druck. Es gilt:<br>
$T \hbox{ konst. } \Rightarrow V \propto \frac{1}{p}$
</p>

<p><b>2. Gay-Lussac'sches Gesetz: </b>Wird eine feste Menge eines idealen Gases auf einem konstanten Volumen V gehalten, während sich die Temperatur oder der Druck ändern, so ist der Druck proportional zur Temperatur T. Es gilt:<br>
$V \hbox{ konst. } \Rightarrow p \propto T$</p>

Für eine feste Menge eines idealen Gases gilt nun: <br>
$
\begin{align*}
&(1)p \propto \frac{1}{V} \\
&(2)p \propto T\\
&\hbox{Daraus folgt: }\\
&p \propto \frac{T}{V} \Rightarrow \frac{p \cdot V}{T} = \hbox{ konst.}
\end{align*} 
$<br><br>
Wird anschließend noch die Teilchenzahl variiert, so ergibt sich die ideale Gasgleichung: <br>
$p \cdot V = N \cdot k_B \cdot T$



<h3> Das Modell des idealen Gases </h3>
Die oben beschriebenen Gesetze setzen voraus, dass mit einem idealen Gas gearbeitet wird. Was ein ideales Gas überhaupt ist, und wir ein Gas in einer Simulation umgesetzt werden kann, wird nun erläutert.

Folgende Annahmen beschreiben das Modell des idealen Gases vollständig:
<ul>
    <li>Atome/Moleküle werden durch kleine starre Kugeln mit Radius $r_0$ approximiert</li>
    <li>N Kugeln existieren in einer Box, Position, Geschwindigkeit, Masse und Radius sind gekannt</li>
    <li>Geschwindigkeit ist statistisch verteilt </li>
    <li>Kugeln interagieren nur durch elastische Stöße mit der Wand oder anderem Teilchen</li>
    <li>$r_0$ ist klein gegenüber mittleren Abstand der Teilchen
</ul>

<table>
  <tr>
    <td style="background-color:white">
        <figure>
          <img src="images/video1.gif" alt="Trulli" style="width:100%">
          <figcaption>elastischer Stoß zwischen Kugel und Wand</figcaption>
        </figure>
    </td>
    <td style="background-color:white">
        <figure>
          <img src="images/video2.gif" alt="Trulli" style="width:100%">
          <figcaption>elastischer Stoß zwischen zwei Kugeln</figcaption>
        </figure>
    </td>
  </tr>
</table>

<h3>Die Mathematik der Stöße</h3>
Stöße sind die einzige Möglichkeit, wie Teilchen miteinander interagieren können. Deswegen ist sie auch das Herzstück der Simulation.

<p><b>Mathematik von elastischen Stößen in 1D: </b>Beim idealen elastischen Stoß stoßen zwei Körper aufeinander ohne dass kinetische Energie verloren geht. Es gilt also der Energie- und Impulserhaltungssatz:<br><br>
$
\begin{align*}
   &\sum E_{kin}=\sum E_{kin}'\\
   &\frac{m_1}{2} \cdot v_1^2+\frac{m_2}{2} \cdot v_2^2 = \frac{m_1}{2} \cdot {v_1'}^2
   + \frac{m_2}{2} \cdot {v_2'}^2\\
   &\frac{m_1}{2} \cdot (v_1^2-{v_1'}^2)=\frac{m_2}{2} \cdot ({v_2'}^2-v_2^2)\\\\
   &\sum \vec{p}= \sum \vec{p'}\\
   &m_1 \cdot \vec{v_1}+m_2 \cdot \vec{v_2} = m_1 \cdot \vec{v_1'}
   + m_2 \cdot \vec{v_2'}\\
   &m_1 \cdot (\vec{v_1}-\vec{v_1'})=m_2 \cdot (\vec{v_2'}-\vec{v_2})\\
\end{align*}
$<br><br>
Durch geeignetes Umstellen (Verweis auf Demtröder) ergibt sich für $v_1'$ und $v_2'$:<br>
    
$
\begin{align*}
    &v_1'=2 \frac{m_1 v_1+m_2 v_2}{m_1 + m_2}-v_1\\
    &v_2'=2 \frac{m_1 v_1+m_2v_2}{m_1+m_2}-v_2
\end{align*}
$
 
    
Für den Fall $m_2 \to \infty$ und $v_2=0$, also eine Wand, gilt:<br>
$
\begin{align*}
    &v_1'=-v_1\\
    &v_2'=0
\end{align*}
$    
</p>

<p><b>Mathematik von Stößen in 2D:</b> Der zweidimensionale elastische Stoß beruht prinzipiell auf dem eindimensionalen Stoß. Für die Berechnung benötigen wir ein geeignetes Koordinatensystem, welches eine Achse senkrecht und eine Achse parallel zur Zentralsteigung besitzt. Die Zentralsteigung beschreibt hierbei die Gerade durch die Mittelpunkte der Kugel. Beim Stoß erfolg die Berechnung wie beim eindimensionalen, aber nur in Richtung der Zentralsteigung.<br>
    
<figure style="float:left;">
          <img src="./images/asset6.svg", width=300px style="margin:10px">
          <figcaption>elastischer Stoß zwischen Kugel und Wand</figcaption>
        </figure>    
<p>
    Für den elastischen Stoß gilt dann:<br>
        $
        \begin{align*}
        &\vec{v_1} =
        \begin{pmatrix}
        v_{1x}\\
        v_{1y}
        \end{pmatrix},
        \vec{v_2} =
        \begin{pmatrix}
        v_{2x}\\
        v_{2y}
        \end{pmatrix}\\
        &\vec{v_1'}=
        \begin{pmatrix}
        2 \frac{m_1 v_{1x}+m_2 v_{2x}}{m_1 + m_2}-v_{1x}\\
        v_{1y}
        \end{pmatrix}\\
        &\vec{v_2'}=
        \begin{pmatrix}
        2 \frac{m_1 v_{1x}+m_2 v_{2x}}{m_1 + m_2}-v_{2x}\\
        v_{2y}
        \end{pmatrix}
        \end{align*}
        $
</p>
    
    


    

<p><b>Koordinatentransformation: </b> Die wichtigste Voraussetzung für die Berechnung zweidimensionaler Stöße ist das geeignete Koordinatensystem, welches nicht trivial gegeben ist. Für die Simulation wird die kanonische Basis verwendet. Um auf unser neues Koordinatensystem zu kommen, nutzen wir eine Basistransformationsmatrix U. Da U nur ein gedrehtes kanonisches Koordinatensystem ist, kann man die Transformationsmatrix als Drehmatrix auffassen.

<figure style="float:left;">
    <img src="./images/asset7.svg" width=300px style="margin:10px"
    <figcaption>elastischer Stoß zwischen Kugel und Wand</figcaption>
</figure>

<p>
    $
    \begin{align*}
        &U_{\alpha} = 
            \begin{pmatrix}
                &\cos(\alpha) &-\sin(\alpha)\\
                &\sin(\alpha) & \cos(\alpha)
            \end{pmatrix}
    \end{align*}
    $<br>
    Möchte man von der neuen Basis zurück in die kanonische gelangen, so berechnet man die Inverse. Für die Rotationsmatrix gilt dann:<br>
    $
    \begin{align*}
        &U_{\alpha}^{-1} = 
            \begin{pmatrix}
                &\cos(-\alpha) &-\sin(-\alpha)\\
                &\sin(-\alpha) & \cos(-\alpha)
            \end{pmatrix}
        \end{align*}
    $<br>
    Ausschlaggebend für die Berechnung der Rotationsmatrix ist der Winkel $\alpha$. Dieser kann über die atan2-Funktion berechnet werden:<br>
    $
    \begin{align*}
        &\alpha = \arctan2(dy,dx) \\
    \end{align*}
    $
</p>

<h2>Results and Discussion</h2>
Nun ist die nötige Theorie bekannt, um die Simulation eines idealen Gases zu schreiben. Diese besteht aus zwei Klassen. Einmal die <i>particle</i> und einmal die <i>simulation</i> Klasse. Im Folgenden werden die Klassen kurz beschrieben:

<h3>particle class</h3>
Die Klasse Partikel dient dazu, die Eigenschaften und Methoden eines Partikels festzuhalten.

<p><b>Die Klasse besitzt folgende Eigenschaften: </b>
<ul>
    <li>Masse m (kg)</li>
    <li>Position p (m)</li>
    <li>Grundgeschwindigkeit v (m/s) </li>
    <li>Kugelradius r (m) </li>
    <li>Ratio</li>
</ul>
Die ersten 4 Eigenschaften sind selbsterklärend, die letzte Eigenschaft beschreibt den Anteil der Grundgeschwindigkeit v, die die Partikel haben (Wenn Ratio=0.5 hat das Teilchen die Hälfte der Grundgeschwindigkeit)

<p><b>Die Klasse besitzt folgende Methoden: </b>
<ul>
    <li><i>move</i> Bewegt das Teilchen entsprechend des Zeitintervalls dt und seiner Geschwindigkeit</li>
    <li><i>bounceWall</i> Korrigiert Geschwindigkeit und Position, falls ein Teilchen mit einer Wand überschneidet</li>
    <li><i>bounce</i> Korrigiert Geschwindigkeit und Position, falls zwei Teilchen kollidieren</li>
    <li><i>getKineticEnergy</i> liefert die kinetische Energie des Teilchens</li>
</ul>
</p>

In [ ]:
class particle:
    
    
    def rotMat(alpha, x):
        return np.array([[np.cos(alpha), -np.sin(alpha)], [np.sin(alpha), np.cos(alpha)]]).dot(x)
    
    def __init__(self, m, x,y, vx, vy,r, color):
        self.m = m     #kg
        self.vx = vx   #m/s
        self.vy = vy   #m/s
        self.x = x     #m
        self.y = y     #m
        self.r = r     #m
        self.ratio = 1
        self.color = color
    
    def move(self, dt, ratio):
            self.x = self.vx * ratio * dt+ self.x
            self.y = self.vy * ratio * dt + self.y
            self.ratio = ratio
            
    def bounceWall(self,n):
        match n:
            case 0:
                self.vx = -self.vx
                self.x = 2*self.r - self.x
            case 1:
                self.vx = -self.vx
                self.x= 2*(simulation.width.value*10**-12-self.r)-self.x
            case 2:
                self.vy = -self.vy
                self.y = 2*self.r-self.y
            case 3:
                self.vy = -self.vy
                self.y = 2*(simulation.height-self.r)-self.y
                
    def bounce(self,p):
        dx = self.x-p.x
        dy = -(self.y-p.y)
        d = np.sqrt(dx**2+dy**2)
        alpha = np.arctan2(dy,dx)
        
        p1=np.array([self.x,self.y])
        p2=np.array([p.x,p.y])
        p1s=particle.rotMat(alpha,p1)
        p2s=particle.rotMat(alpha,p2)
        p1ns=np.array([p1s[0]+(2*self.r-d)/2,p1s[1]])
        p2ns=np.array([p2s[0]-(2*self.r-d)/2,p2s[1]])
        p1n=particle.rotMat(-alpha,p1ns)
        p2n=particle.rotMat(-alpha,p2ns)
        self.x=p1n[0]
        self.y=p1n[1]
        p.x=p2n[0]
        p.y=p2n[1]
         
        v1 = np.array([self.vx, self.vy])
        v2 = np.array([p.vx, p.vy])
        v1s = particle.rotMat(alpha,v1)
        v2s = particle.rotMat(alpha,v2)
        v1ns=(v2s[0],v1s[1])
        v2ns=(v1s[0],v2s[1])
        v1n= particle.rotMat(-alpha,v1ns)
        v2n= particle.rotMat(-alpha,v2ns)
        self.vx = v1n[0]
        self.vy = v1n[1]
        p.vx = v2n[0]
        p.vy = v2n[1]
        
    def getKineticEnergy(self,dt):
        return 1/2 * self.m * ((self.vx*self.ratio)**2+(self.vy*self.ratio)**2)

<h3>simulation class</h3>
Die Klasse dient dazu, Stöße zu registrieren, Partikel zu bewegen, visualisieren und statistische Größen (Temperatur, Druck, Volumen) auszugeben.

In [ ]:
class simulation:
    
    p = []
    height = 3000e-12        #m (3000pm)
    dt = 20e-15              #s (200fs)
    radius = 20e-12          #m (50pm)
    k = 1.380649 *10**-23    #J/K
    l = []
    t = 0
    impulse = np.zeros(50)
    E_kin = 0
    
    startSimulation=False
    
    veloRatio = widgets.FloatSlider(min=0.1,max=3, value=1,step=0.01, description="Velocity")
    width = widgets.IntSlider(min=500, max= 10000, value=10000, description="Width (pm) ")
    buttonAdd = widgets.Button(description="+")
    buttonRem = widgets.Button(description="-")
    buttonStart = widgets.Button(description="start")
    buttonStop = widgets.Button(description="stop")
    textVolume = widgets.FloatText(description="Volume (pm³)", disabled=True)
    textPressure = widgets.FloatText(description="Pressure", disabled=True)
    textTemperature = widgets.FloatText(description="Temperature", disabled=True)
    textParticles = widgets.IntText(disabled = True)
    canvas = Canvas(width = width.max/10, height=height*10**12/10)
    
    def generateParticles(button=widgets.Button(), n=10):
        for _ in range(n):
            v = random.randint(0,2000)
            alpha = random.random() * 2 * np.pi
            simulation.p.append(particle(
                2.6569e-26,
                random.randint(0,simulation.width.value)*10**-12,
                random.randint(0,simulation.height*10**12)*10**-12,
                v * np.cos(alpha),
                v * np.sin(alpha),
                simulation.radius,
                "black"))
            simulation.textParticles.value = len(simulation.p)
                
            
    def generateParticlesUniformly(n=10, v = 1000):
        for _ in range(n):
            alpha = random.random() * 2 * np.pi
            simulation.p.append(particle(
                2.6569e-26,
                random.randint(0,simulation.width.value)*10**-12,
                random.randint(0,simulation.height*10**12)*10**-12,
                v * np.cos(alpha),
                v * np.sin(alpha),
                simulation.radius,
                "black"))
            simulation.textParticles.value = len(simulation.p)
    
    def removeParticles(button=widgets.Button(), n=10):
            for _ in range(n):
                if simulation.p:
                    simulation.p.pop()
            simulation.textParticles.value = len(simulation.p)

    def deleteParticles():
        simulation.p = []
    
    def addParticle(particle):
        simulation.p.append(particle)
    
    def reset():
        simulation.width.value=4000
        simulation.veloRatio.value=1
        simulation.t = 0
        simulation.deleteParticles()
        simulation.stop()

    def collisionTest():
        count = 1
        for i in simulation.p:
            
            for j in simulation.p[count::]:
                dx = j.x-i.x
                dy = j.y-i.y
                if dx**2+dy**2 < (i.r+j.r)**2:
                     if (dx*i.vx <=0 and dx*j.vx >=0 ) and (dy*i.vy <=0 and dy*j.vy >=0 ):
                        pass 
                     else:
                        i.bounce(j)
            count = count + 1
            
            if i.x < i.r:
                i.bounceWall(0)
                simulation.impulse[0] = simulation.impulse[0] + abs(2 * i.m * i.vx * i.ratio)
            elif i.x > simulation.width.value*10**-12-i.r:
                i.bounceWall(1)
                simulation.impulse[0] = simulation.impulse[0] + abs(2 * i.m * i.vx * i.ratio)
                
            if i.y < i.r:
                i.bounceWall(2)
                simulation.impulse[0] = simulation.impulse[0] + abs(2 * i.m * i.vy * i.ratio)
            elif i.y > simulation.height-i.r:
                i.bounceWall(3)
                simulation.impulse[0] = simulation.impulse[0] + abs(2 * i.m * i.vy * i.ratio)
            
            simulation.E_kin = simulation.E_kin + i.getKineticEnergy(simulation.dt)
                
            
                            
    def animateParticles():     #1px = 10pm
        while simulation.startSimulation:
            with hold_canvas():
                simulation.canvas.clear()
                simulation.canvas.stroke_rect(0,0,width=simulation.width.value/10,height=simulation.height*10**12/10)
                for i in simulation.p:
                    simulation.canvas.fill_style = i.color
                    simulation.canvas.fill_circle(i.x*10**12/10, i.y*10**12/10, i.r*10**12/10)
                    i.move(simulation.dt, simulation.veloRatio.value)
            
            simulation.collisionTest()
            simulation.updateValues()
            sleep(simulation.dt*10**12)

        
    def updateValues():
            simulation.textVolume.value=simulation.width.value*10**(-12) * simulation.height * 2 *simulation.radius
            simulation.textPressure.value = np.mean(simulation.impulse)/(simulation.dt * 4 *simulation.radius *(simulation.width.value*10**(-12) +simulation.height))
            simulation.impulse = np.roll(simulation.impulse,1)
            simulation.impulse[0] = 0
            simulation.t = simulation.t+ simulation.dt
            if simulation.p:
                simulation.E_kin = simulation.E_kin/len(simulation.p)
                simulation.textTemperature.value= simulation.E_kin/simulation.k
            simulation.E_kin = 0
                
                
    def initialize():
        simulation.buttonAdd.on_click(simulation.generateParticles)
        simulation.buttonRem.on_click(simulation.removeParticles)
        simulation.buttonStart.on_click(simulation.start)
        simulation.buttonStop.on_click(simulation.stop)
        
        items = [widgets.HBox([simulation.buttonAdd, simulation.buttonRem, simulation.textParticles]),
                 simulation.textVolume, simulation.buttonStart, 
                 simulation.width, simulation.textPressure, simulation.buttonStop, 
                 simulation.veloRatio, simulation.textTemperature]
        w = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))
        
        display(w)
        display(simulation.canvas)
        
    def show():
        simulation.startSimulation=False
        display(simulation.canvas)
        simulation.start()
        
            
    def start(button=widgets.Button()):
        simulation.startSimulation = True
        t = Thread(target=simulation.animateParticles, daemon=True)
        t.start()
        t = 0
        
    def stop(button=widgets.Button()):
        simulation.startSimulation = False
        
         

<h3>Grundannahmen</h3>

<h4>statistische Verteilung der Geschwindigkeit: </h4>
Eine wichtige Voraussetzung für ein ideales Gas ist die Isotropie der Geschwindigkeitsverteilung. Diese wird nun überprüft.

Es werden 200 Partikel generiert und deren Geschwindigkeit in zwei Arrays gespeichert. Anschließend werden die quadratisch gemittelte Gesamtgeschwindigkeit und die quadratisch gemittelten Geschwindigkeiten in x und y Richtung gebildet. 

In [ ]:
simulation.deleteParticles()
simulation.generateParticles(n=200)
vx = np.array([])
vy = np.array([])
for i in simulation.p:
    vx = np.append(vx, i.vx)
    vy = np.append(vy, i.vy)
    
v_mean = np.mean(np.sqrt(vx**2+vy**2))

v_squared_mean = np.mean(vx**2+vy**2)
vx_squared_mean = np.mean(vx**2)
vy_squared_mean = np.mean(vy**2)

Schauen wir uns die Werte an:

In [ ]:
plt.xlabel("v in m/s")
plt.ylabel("Anzahl")
plt.hist(np.sqrt(vx**2+vy**2),10)
plt.show()

print(f"mittlere Geschwindigkeit: {v_mean} m/s")
print(f"quadratisch gemittelte Geschindigkeit: {v_squared_mean} m²/s²")
print(f"quadratisch gemittelte Geschwindigkeit in x-Richtung: {vx_squared_mean} m²/s²")
print(f"quadratisch gemittelte Geschwindigkeit in y-Richtung: {vy_squared_mean} m²/s²" )

Wir erkennen, dass folgende Beziehungen gelten: <br>
$
\begin{align}
    \overline{{v_x}^2} \approx \frac{1}{2} \cdot \overline{v²} \\
    \overline{{v_y}^2} \approx \frac{1}{2} \cdot \overline{v²}
\end {align}
$ <br>
Die Geschwindigkeitsverteilung ist also isotrop.

<h4>Stöße</h4>

<p>Jetzt ist es an der Zeit, die Stöße zu testen. Führe dafür zuerst <i>simulation.show()</i> aus und dann das gewünschte Preset</p>

In [ ]:
simulation.reset()
simulation.show()

<b>Preset 1 - Testkollision mit der Wand</b>

In [ ]:
simulation.deleteParticles()
simulation.addParticle(particle(1,500*10**-12,500*10**-12,1000,1000,30*10**-12,"black"))

<b>Preset 2 - Horizontaler Stoß</b>

In [ ]:
simulation.deleteParticles()
simulation.addParticle(particle(1,250*10**-12,1500*10**-12,1000,0,30*10**-12,"blue"))
simulation.addParticle(particle(1,600*10**-12,1500*10**-12,-500,0,30*10**-12,"red"))

<b>Preset 3 - vertikaler Stoß<br>

In [ ]:
simulation.deleteParticles()
simulation.addParticle(particle(1,2000*10**-12,1000*10**-12,0,0,30*10**-12,"blue"))
simulation.addParticle(particle(1,2000*10**-12,2000*10**-12,0,1000,30*10**-12,"red"))

<b>Preset 4 - schiefer Stoß</b>

In [ ]:
simulation.deleteParticles()
simulation.addParticle(particle(1,250*10**-12,1500*10**-12,1000,0,30*10**-12,"blue"))
simulation.addParticle(particle(1,2000*10**-12,1510*10**-12,-500,0,30*10**-12,"red"))

<b>Preset 5 - Billard Stoß </b>

In [ ]:
simulation.deleteParticles()
simulation.addParticle(particle(1,250*10**-12,1500*10**-12,1000,0,30*10**-12,"blue"))
simulation.addParticle(particle(1,2500*10**-12,1500*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2560*10**-12,1470*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2560*10**-12,1530*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2620*10**-12,1500*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2620*10**-12,1440*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2620*10**-12,1560*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2680*10**-12,1530*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2680*10**-12,1590*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2680*10**-12,1470*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2680*10**-12,1410*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2740*10**-12,1500*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2740*10**-12,1560*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2740*10**-12,1620*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2740*10**-12,1440*10**-12,0,0,30*10**-12,"black"))
simulation.addParticle(particle(1,2740*10**-12,1380*10**-12,0,0,30*10**-12,"black"))

<b>Preset 6 - Your Choice</b>

In [ ]:
simulation.deleteParticles()
#add your code here. Feel free to explore

<h3>Herleitung fundamentaler Zusammenhänge</h3>

Jetzt kommen wir abschließend zur Herleitung der idealen Gasgleichung. Dazu zeigen wir zuerst das Gesetz von Boyle-Marionette.

<h4>Boyle-Marionette</h4>

Um das Gesetz von Boyle-Marionette numerisch zu beweisen, generieren wir zuerst 200 Partikel und starten die Simulation. Während wir aller 3 Sekunden das Volumen unserer Box verkleinern, werden in zwei Arrays Volumen und Druck gemessen und aufgezeichnet. Dabei ist die Temperatur konstant. Anschließend werden die Daten als Scatterplot ausgegeben, um einen möglichen Zusammenhang festzustellen.

In [ ]:
simulation.reset()
simulation.width.value=10000
simulation.generateParticles(n=200)


volume = np.array([])
pressure = np.array([])

simulation.show()


for i in range(0,9000,200):
    simulation.width.value=10000-i
    sleep(3)
    volume=np.append(volume, simulation.width.value*10**-12*simulation.height*2*simulation.radius)
    pressure=np.append(pressure, simulation.textPressure.value)

simulation.stop()
plt.scatter(pressure,volume)
plt.xlabel(f"Pressure in Pa")
plt.ylabel(f"Volume in $m^3$")
plt.show()

Der gezeigte Plot sieht eindeutig nach einer Hyperbel aus. Dann müsste gelten: $V=\frac{a}{p}$ Um unsere These zu bestätigen, führen wir eine Regression aus. 

In [ ]:
def hyperbola(x,a):
    return a/x

a, cov =curve_fit(hyperbola,pressure,volume,p0=10**-18,absolute_sigma=True)
plt.scatter(pressure, volume, color="green", label="data", alpha=0.8)
plt.plot(pressure, hyperbola(pressure, a),color="tab:blue", label="theory")
plt.legend()
plt.xlabel(f"Pressure in Pa")
plt.ylabel(f"Volume in $m^3$")
plt.show()
print(f"a = {a}")

Wir haben also festgehalten, dass es sich hier um einen hyperbolischen Zusammenhang handelt. Mathematisch bedeutet das:<br>
$\begin{align*}
    &V = \frac{a}{p}\\
    \Rightarrow &V \propto \frac{1}{p}
\end{align*}$

Wir wissen also nun durch unsere Simulation, dass das Gesetz von Boyle-Marionette stimmt!

<b>2. Gay-Lussac'sches Gesetz: </b>

Als nächstes widmen wir uns dem 2. Gesetz von Gay-Lussac. Um einen numerischen Beweis zu liefern, generieren wir wieder 200 Partikel und starten die Simulation. Bei festem Volumen wird nun aller 3 Sekunden die Geschwindigkeit der Teilchen (und somit die Temperatur) erhöht. Dabei werden in zwei Arrays Druck und Temperatur aufgezeichnet und anschließend als Scatterplot ausgegeben, um einen möglichen Zusammenhang zu finden.

In [ ]:
simulation.reset()
simulation.width.value=4000
simulation.generateParticles(n=200)
simulation.show()

pressure = np.array([])
temperature = np.array([])
for i in range(10,150,5):
    simulation.veloRatio.value=i/100
    sleep(3)
    temperature=np.append(temperature,simulation.textTemperature.value)
    pressure=np.append(pressure, simulation.textPressure.value)

simulation.stop()
plt.scatter(temperature,pressure)
plt.xlabel("Temperature in K")
plt.ylabel("Pressure in Pa")
plt.show() 

Der gezeigte Plot sieht sehr nach einer linearen Funktion aus. Dann muss gelten: $p = m \cdot T$. Überprüfen wir es mithilfe einer Regression:

In [ ]:
def linear(x,a):
    return a*x

m, cov=curve_fit(linear,temperature,pressure,absolute_sigma=True)
plt.scatter(temperature, pressure, color="green", label="data", alpha=0.8)
plt.plot(temperature, linear(temperature, m), color="tab:blue", label="theory")
plt.xlabel(f"Temperature in K")
plt.ylabel(f"Pressure in Pa")
plt.legend()
plt.show()
print(f"m = {m}")

Wir können also bestätigen, dass es sich um einen linearen Zusammenhang handelt. das bedeutet mathematisch:<br>
$
\begin{align*}
&p = m \cdot T\\
\Rightarrow& p \propto T
\end{align*}
$<br>
Also stimmt auch das 2. Gay-Lussac'sche Gesetz!

<b>allgemeines Gasgesetz</b>

Mithilfe der beiden obigen Gesetze gilt: $\frac{p \cdot V}{T} = konst.$ für eine feste Menge an idealem Gas. Dies wollen wir kurz überprüfen. Dafür generieren wir wieder 200 Teilchen, starten die Simulation und variieren abwechselnd Volumen und Geschwindigkeit. Dabei wird Druck, Temperatur und Volumen in drei verschiedenen Arrays gemessen und aufgezeichnet und in einem Scatterplot der Quotient $\frac{p \cdot V}{T}$ über die Zeit aufgetragen.

In [ ]:
simulation.reset()
simulation.width.value=10000
simulation.generateParticles(n=200)
simulation.show()

time = np.array([])
pressure = np.array([])
temperature = np.array([])
volume = np.array([])

for i in range(0,9000,500):
    for j in range (1,102,25):
        simulation.width.value=10000-i
        simulation.veloRatio.value=j/100
        sleep(3)
        temperature=np.append(temperature,simulation.textTemperature.value)
        pressure=np.append(pressure, simulation.textPressure.value)
        volume=np.append(volume, simulation.width.value*10**-12*simulation.height*2*20.10**-12*2*simulation.radius)
        time=np.append(time, simulation.t)

simulation.stop()
plt.plot(time,pressure*volume/temperature)
plt.xlabel(f"time in s")
plt.ylabel(f"Quotient \frac{p \cdot V}{T} in \frac{m^3 \cdot Pa}{K}")
plt.show()

Es ist leicht zu erkennen, dass der Wert um eine konstante schwankt, was unsere Theorie ebstätigt.

<b>ideales Gasgesetz</b>

Um nun abschließend das ideale Gasgesetz numerisch zu beweisen, generieren wir 50 Teilchen und halten Volumen und Temperatur konstant. Indem wir jeweils nach einer Sekunde die Teilchenanzahl um 1 erhöhen und sie zusammen mit dem Druck in zwei Arrays aufzeichnen, können wir einen Scatterplot auftragen, um so den Zusammenhang zu finden.

In [ ]:
simulation.reset()
simulation.generateParticlesUniformly(n=50, v=1000)
simulation.show()

pcount = np.array([])
pressure = np.array([])

for i in range(50,300,1):
    pcount = np.append(pcount, simulation.textParticles.value)
    pressure = np.append(pressure, simulation.textPressure.value)
    sleep(1)
    simulation.generateParticlesUniformly(n=1, v=1000)
    
plt.scatter(pcount,pressure)
plt.xlabel("particle count (absolute Value)")
plt.ylabel("Pressure in Pa")
plt.show()

In [ ]:
simulation.stop()